In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ovm.utils import (
    fetch_data, 
    convert_multiple_price_histories_from_nested_lists_to_dict_of_dataframes, 
    convert_price_history_from_nested_list_to_dataframe, 
    compute_number_of_days_in_price_history, 
    save_price_histories
)

from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import typing as tp

In [3]:
def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)